<h1>Author: Blake Conrad</h1>
<h2>Simple IPython Notebook with the hope of:</h2>
<h3>Querying the bible from a local MySQL database</h3>

<h3>The following were the necessary installer commands, or packages I needed to set up MySQL with Python</h3>

<ul>
<li>export PATH=$PATH:/usr/local/mysql/bin</li>
<li>sudo easy_install mysql-python</li>
<li>pip install mysql-python</li>
<li>pip install mysqlclient</li>
<li>Or ... however you have to install the package: 
<ul><li>import MySQLdb</li> </ul>
</li>


</ul>



In [197]:
# Query the local database

hostname = '127.0.0.1'
username = 'root'
password = 'NICE TRY MATE'
database = 'Bible'

        
# Getting some exposure to the database

print "Using MySQLdb…"
import MySQLdb
myConnection = MySQLdb.connect( host=hostname, user=username, passwd=password, db=database )

d_books = getAllBooks(myConnection, debug=True)
myConnection.close()
    





Using MySQLdb…
Chapters in 1: 50
Chapters in 2: 40
Chapters in 3: 27
Chapters in 4: 36
Chapters in 5: 34
Chapters in 6: 24
Chapters in 7: 21
Chapters in 8: 4
Chapters in 9: 31
Chapters in 10: 24
Chapters in 11: 22
Chapters in 12: 25
Chapters in 13: 29
Chapters in 14: 36
Chapters in 15: 10
Chapters in 16: 13
Chapters in 17: 10
Chapters in 18: 42
Chapters in 19: 150
Chapters in 20: 31
Chapters in 21: 12
Chapters in 22: 8
Chapters in 23: 66
Chapters in 24: 52
Chapters in 25: 5
Chapters in 26: 48
Chapters in 27: 12
Chapters in 28: 14
Chapters in 29: 3
Chapters in 30: 9
Chapters in 31: 1
Chapters in 32: 4
Chapters in 33: 7
Chapters in 34: 3
Chapters in 35: 3
Chapters in 36: 3
Chapters in 37: 2
Chapters in 38: 14
Chapters in 39: 4
Chapters in 40: 28
Chapters in 41: 16
Chapters in 42: 24
Chapters in 43: 21
Chapters in 44: 28
Chapters in 45: 16
Chapters in 46: 16
Chapters in 47: 13
Chapters in 48: 6
Chapters in 49: 6
Chapters in 50: 4
Chapters in 51: 4
Chapters in 52: 5
Chapters in 53: 3
Chapt

In [198]:
d_books["Genesis"][1]

['In the beginning God created the heaven and the earth.',
 'And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.',
 'And God said, Let there be light: and there was light.',
 'And God saw the light, that it was good: and God divided the light from the darkness.',
 'And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.',
 'And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters.',
 'And God made the firmament, and divided the waters which were under the firmament from the waters which were above the firmament: and it was so.',
 'And God called the firmament Heaven. And the evening and the morning were the second day.',
 'And God said, Let the waters under the heaven be gathered together unto one place, and let the dry land appear: and it was so.',
 'And God called the dry land Eart

In [195]:
"""
Function: getAllChapters
Args: 
    1. Pre-existing Database connection (MySQLdb object reference)
    2. Arbitrary book number
        - Ex) Genesis == 1,
              Exodus == 2, 
              ... ,
              Judge == 65, 
              Revelation == 66
              
Usuage: Meant to be utility function and used in a loop
Purpose: 
    1. Recieve a db connection and a book number
    2. Execute a SQL Query to count the number of chapters in a book
    3. Loop all chapters running another SQL query to go get the chapter contents
    4. Convert the chapter contents from list of tuples to list of strings
    5. Return the d_chapters dictionary
    
Return: 
    - Python dictionary object named d_chapters
    - (Example) d_chapters[1] == ["verse1","verse2",..."verseN"]
"""

def getAllChapters(conn, bookNumber, debug=False):
    
    import re
    cur = conn.cursor()
    
    queryChapterCount = "call sp_getChapterCountInBook(" +str(bookNumber) + ");"
    cur.execute(queryChapterCount)
    
    # gross ...
    chapCount = str(re.findall(r'\w+', str(cur.fetchall()[0]))[0].replace('L',''))
    
    # THIS IS THE NEXT FIX, NOT A BIG ONE ONCE YOU TAKE A LOOK AT IT.
    
    if debug: 
        print "Chapters in " + str(bookNumber) + ": " + chapCount
    
    
    every_chapter_id = [i for i in range(1,len(chapCount)+1)]
    
    s_mainQueryString = "CALL sp_getBookByChapter"
    s_argsString = ""
    
    d_chapters = {}
    for chap in range(1,int(chapCount)+1):
        s_argsString = "("+str(bookNumber)+","+str(chap)+")"
        s_tmpQueryString = s_mainQueryString + s_argsString
        #print s_tmpQueryString
        cur.execute(s_tmpQueryString)
        d_chapters[chap] = [ "%s" % x for x in cur.fetchall()] 

        
    return d_chapters

In [196]:
"""
Function: getAllBooks
Args: 
    1. Pre-existing Database connection (MySQLdb object reference)
    2. (Optional) Set your debug = True as an argument and it will print out updates
        + (Example) d_books = getAllBooks(myConnection, debug=True)
        
Usuage: Standalone function build to return a clean bible dictionary object
Purpose: 
    1. Create the book ID's (tons of book titles)
    2. Loop through each book title
        a. Go get the chapters associated with that book
    3. Return our d_books dictionary object
    
Return: 
    - Python dictionary object named d_books
    - (Example) 
        + d_books["Genesis"][1] returns a list of strings representing
            Genesis chapter 1
        + d_books["John][13][13] returns a single string representing
            John chapter 13 verse 13
"""

def getAllBooks(conn, debug=True):
    
    # Yes, I took the time to go gather this information by hand
    # https://en.wikipedia.org/wiki/Books_of_the_Bible
    bookNames = ["Genesis", 
                 "Exodus", 
                 "Leviticus",
                 "Numbers", 
                 "Deuteronomy",
                "Joshua", 
                 "Judges",
                 "Ruth",
                 "1 Samuel",
                 "2 Samuel",
                 "1 Kings", 
                 "2 Kings", 
                 "1 Chronicles",
                 "2 Chronicles",
                "Ezra",
                 "Nehemiah",
                 "Esther",
                "Job", 
                 "Psalms", 
                 "Proverbs",
                 "Ecclesiates", 
                 "Song of Solomon",
                "Isaiah",
                 "Jeremiah",
                 "Lamentations", 
                 "Ezekiel",
                 "Daniel",
                "Hosea", 
                 "Joel",
                 "Amos", 
                 "Obadiah",
                 "Jonah",
                 "Micah",
                 "Nahum",
                "Habakkuk", 
                "Zephaniah",
                 "Haggai",
                 "Zechariah",
                 "Malachi",
                "Matthew",
                 "Mark", 
                 "Luke", 
                 "John", 
                 "Acts",
                 "Romans",
                "1 Corinthians",
                 "2 Corinthians", 
                 "Galatians", 
                 "Ephesians",
                "Philippians",
                 "Colossians", 
                 "1 Thessalonians",
                 "2 Thessalonians",
                "1 Timothy",
                 "2 Timothy",
                 "Titus", 
                 "Philemon",
                 "Hebrews", 
                 "James", 
                 "1 Peter",
                 "2 Peter",
                 "1 John", 
                 "2 John", 
                 "3 John",
                "Jude", 
                 "Revelation"]


    d_books = {}
    count=1
    for book in bookNames:
        
        if debug: print "Collecting " , book ,"'s chapters."
        d_books[book] = getAllChapters(conn, count, debug)
        count+=1
    
    return d_books

